In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
import numpy as np
import typing
import matplotlib.pyplot as plt
from numpy.lib.stride_tricks import sliding_window_view
import time
import tensorflow as tf
import keras
import  math


KeyboardInterrupt



### Define Vars

In [2]:
input_sequence_length = 12
forecast_horizon = 12     # 24 = 2 hr,    12 = 1 hr,     6 = 30 min
# path = "D:\MyData/Extracted Data/train"
# path = "D:\MyData/Clustered Data/train/4"
path = "D:\MyData/Extracted Data/train"
dir_list = os.listdir(path)
as_strided = np.lib.stride_tricks.as_strided  
num_files = len(dir_list)
features = 13
batch_size = 64


In [3]:
start_time = time.time()
targets_array = []
inputs_array = []

for file in dir_list:
    filepath = path + '/' + file
    cur_data = pd.read_csv(filepath)
    cur_data.drop(columns=['EventID', 'FileName'], inplace=True)
    # normalized_cur_data = (cur_data - cur_data.min()) / (cur_data.max() - cur_data.min())
    normalized_cur_data=(cur_data-cur_data.mean())/cur_data.std()
    ncd = normalized_cur_data.to_numpy()
    target_start = input_sequence_length + forecast_horizon
    win = (input_sequence_length, ncd.shape[1])
    # temp = (ncd.shape[0] - (forecast_horizon+input_sequence_length+1))
    input_v1 = sliding_window_view(ncd[:(ncd.shape[0]-forecast_horizon-1)], win, axis=(0, 1))
    inputs = input_v1[:,0,:,:]
    targets = normalized_cur_data[target_start:]
    targets_array.append(targets)
    inputs_array.append(inputs)
    # targets_array = np.append(targets_array, targets)
    # inputs_array = np.append(inputs_array, inputs)
print("--- %s seconds ---" % (time.time() - start_time))


--- 48.9102520942688 seconds ---


In [4]:
# input = tf.convert_to_tensor(inputs_array)
# input2 = tf.data.Dataset.from_tensors(inputs_array)
# input2
# print(len(inputs_array))
# print(inputs_array[0].shape)
xtrain = [inputs_array]

In [5]:
# inputs_array = inputs_array.reshape(-1, input_sequence_length, features)
# targets_array = targets_array.reshape(-1, features)
# X_Train = tf.convert_to_tensor(inputs_array)
# Y_Train = tf.convert_to_tensor(targets_array)
# dataset = tf.data.Dataset.from_tensor_slices((X_Train, Y_Train))


In [6]:
# dataset = X_Train.batch(64)
# list(dataset.as_numpy_iterator())
# type(Batched_Y_Train)
# print(targets_array)
# print(inputs_array)
# xtrain
# Type(X_Train)
# dataset = tf.data.Dataset.from_tensor_slices(X_Train, Y_Train)
X_Train_Array = np.array(inputs_array)
Y_Train_Array = np.array(targets_array)
# print(X_Train_Array.shape)
X_Train_Array = X_Train_Array.reshape(-1, input_sequence_length, features)
Y_Train_Array = Y_Train_Array.reshape(-1, features)
# batched_dataset = dataset.batch(64)
print(X_Train_Array.shape)
print(Y_Train_Array.shape)
X_Train_Array = np.nan_to_num(X_Train_Array, nan=0, posinf=0, neginf=0)
Y_Train_Array = np.nan_to_num(Y_Train_Array, nan=0, posinf=0, neginf=0)

(261725, 12, 13)
(261725, 13)


In [7]:
# # Define LSTM model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(X_Train_Array.shape[1], X_Train_Array.shape[2])))
model.add(tf.keras.layers.LSTM(32, return_sequences=False, input_shape=(X_Train_Array.shape[1], X_Train_Array.shape[2])))
# model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='tanh'))
# model.add(tf.keras.layers.Dense(13, activation='linear'))
model.add(tf.keras.layers.Dense(13, activation='linear'))
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
model.summary()

C:\Users\jared\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 12, 64)              │          19,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 13)                  │             845 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 35,341 (138.05 KB)

 Trainable params: 35,341 (138.05 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# print(X_Train_Array.shape)
# print(Y_Train_Array.shape)
# transposed_X_Train_Array = np.transpose(X_Train_Array, (0, 2, 1))
# print(transposed_X_Train_Array.shape)


In [9]:
# # # Define LSTM model
# model = tf.keras.models.Sequential()
# # model.add(tf.keras.layers.InputLayer(shape=(12, 13)))
# model.add(tf.keras.layers.LSTM(32, input_shape=(13,1)))
# # model.add(tf.keras.layers.Dense(128, activation='tanh'))
# # model.add(tf.keras.layers.Dense(64, activation='tanh'))
# model.add(tf.keras.layers.Dense(13, activation='linear'))
# # # Compile model
# # model.compile(loss='mse', optimizer='adam')
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# print(model.summary())


In [10]:
model.fit(X_Train_Array, Y_Train_Array, epochs=100,  batch_size=batch_size, verbose=1)

Epoch 1/100
4090/4090 ━━━━━━━━━━━━━━━━━━━━ 68s 14ms/step - loss: 0.8042 - mse: 0.8042
Epoch 2/100
4090/4090 ━━━━━━━━━━━━━━━━━━━━ 53s 13ms/step - loss: 0.7337 - mse: 0.7337
Epoch 3/100
4090/4090 ━━━━━━━━━━━━━━━━━━━━ 51s 13ms/step - loss: 0.7108 - mse: 0.7108
Epoch 4/100
4090/4090 ━━━━━━━━━━━━━━━━━━━━ 102s 25ms/step - loss: 0.6717 - mse: 0.6717
Epoch 7/100
4090/4090 ━━━━━━━━━━━━━━━━━━━━ 92s 22ms/step - loss: 0.6621 - mse: 0.6621
Epoch 8/100
4090/4090 ━━━━━━━━━━━━━━━━━━━━ 139s 22ms/step - loss: 0.6537 - mse: 0.6537
Epoch 9/100
4090/4090 ━━━━━━━━━━━━━━━━━━━━ 97s 24ms/step - loss: 0.6363 - mse: 0.6363
Epoch 13/100
4090/4090 ━━━━━━━━━━━━━━━━━━━━ 127s 20ms/step - loss: 0.6320 - mse: 0.6320
Epoch 14/100
4090/4090 ━━━━━━━━━━━━━━━━━━━━ 156s 23ms/step - loss: 0.6306 - mse: 0.6306
Epoch 15/100
4090/4090 ━━━━━━━━━━━━━━━━━━━━ 65s 16ms/step - loss: 0.6288 - mse: 0.6288
Epoch 16/100
4090/4090 ━━━━━━━━━━━━━━━━━━━━ 56s 14ms/step - loss: 0.6252 - mse: 0.6252
Epoch 17/100
4090/4090 ━━━━━━━━━━━━━━━━━━━━ 59

In [11]:
# path = "D:\MyData/Clustered Data/test/4"
path = "D:\MyData/Extracted Data/test"
dir_list = os.listdir(path)
# as_strided = np.lib.stride_tricks.as_strided  
num_files = len(dir_list)

In [12]:
# start_time = time.time()

# inputs_array = np.empty([0])
# targets_array = np.empty([0])

# for file in dir_list:
#     filepath = path + '/' + file
#     cur_data = pd.read_csv(filepath)
#     cur_data.drop(columns=['EventID', 'FileName'], inplace=True)
#     normalized_cur_data = (cur_data - cur_data.min()) / (cur_data.max() - cur_data.min())
#     ncd = normalized_cur_data.to_numpy()
#     target_start = input_sequence_length + forecast_horizon
#     win = (input_sequence_length, ncd.shape[1])
#     input_v1 = sliding_window_view(ncd[:forecast_horizon], win, axis=(0, 1))
#     inputs = input_v1[:,0,:,:]
#     targets = normalized_cur_data[target_start:]
#     # targets_array.append(targets)
#     # inputs_array.append(inputs)
#     targets_array = np.append(targets_array, targets)
#     inputs_array = np.append(inputs_array, inputs)

In [13]:
targets_array = []
inputs_array = []
# inputs_array = np.empty([num_files*(49-(input_sequence_length + forecast_horizon)), 12, 13])
# inputs_array = np.empty([0])
# targets_array = np.empty([0])

for file in dir_list:
    filepath = path + '/' + file
    cur_data = pd.read_csv(filepath)
    cur_data.drop(columns=['EventID', 'FileName'], inplace=True)
    # normalized_cur_data = (cur_data - cur_data.min()) / (cur_data.max() - cur_data.min())
    normalized_cur_data=(cur_data-cur_data.mean())/cur_data.std()
    ncd = normalized_cur_data.to_numpy()
    target_start = input_sequence_length + forecast_horizon
    win = (input_sequence_length, ncd.shape[1])
    # temp = (ncd.shape[0] - (forecast_horizon+input_sequence_length))
    input_v1 = sliding_window_view(ncd[:(ncd.shape[0]-forecast_horizon-1)], win, axis=(0, 1))
    inputs = input_v1[:,0,:,:]
    targets = normalized_cur_data[target_start:]
    targets_array.append(targets)
    inputs_array.append(inputs)
    # targets_array = np.append(targets_array, targets)
    # inputs_array = np.append(inputs_array, inputs)
print("--- %s seconds ---" % (time.time() - start_time))

--- 10676.99323296547 seconds ---


In [14]:
X_Test = np.array(inputs_array)
Y_Test = np.array(targets_array)
X_Test = X_Test.reshape(-1, input_sequence_length, features)
Y_Test = Y_Test.reshape(-1, features)
print(inputs_array[0].shape)
print(X_Test.shape)
print(Y_Test.shape)
X_Test = np.nan_to_num(X_Test, nan=0, posinf=0, neginf=0)
Y_Test = np.nan_to_num(Y_Test, nan=0, posinf=0, neginf=0)

(25, 12, 13)
(56125, 12, 13)
(56125, 13)


In [15]:
train = Y_Train_Array[:,0]
trainScore = model.evaluate(X_Train_Array, Y_Train_Array, verbose=0)
trainScore
# print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore, math.sqrt(trainScore)))
# testScore = model.evaluate(X_Test, Y_Test, verbose=0)
# print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore, math.sqrt(testScore)))

[0.5748598575592041, 0.5748598575592041]

In [16]:
testScore = model.evaluate(X_Test, Y_Test, verbose=0)
testScore
# print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore, math.sqrt(testScore)))

[0.7248660922050476, 0.7248660922050476]

In [17]:
print(np.min(Y_Train_Array[:,0]))
print(np.max(Y_Train_Array[:,0]))
print(np.var(Y_Train_Array[:,0]))


-5.140256422646281
6.856570193821878
0.9718471223202347


In [29]:
predictions = model.predict(X_Test)

1754/1754 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step


In [30]:
predictions

array([[-0.647942  , -0.18811676, -0.5678177 , ...,  0.34641802,
         0.04437361, -0.07588632],
       [-0.7081665 , -0.20875695, -0.6379456 , ...,  0.35304505,
         0.04960012, -0.10732969],
       [-0.7953945 , -0.12892747, -0.6627957 , ...,  0.5646152 ,
         0.01091696, -0.07538375],
       ...,
       [-1.1797678 , -1.6015683 , -1.4253792 , ...,  0.05739963,
        -0.32472754,  0.20104837],
       [-1.1375659 , -1.507369  , -1.360708  , ...,  0.09063441,
        -0.32724103,  0.2262975 ],
       [-1.6185193 , -1.483084  , -1.5475973 , ...,  0.2299963 ,
        -0.42582205,  0.22465226]], dtype=float32)

In [31]:
Y_Test

array([[-0.41464968, -0.06098757, -0.35015313, ...,  0.71762363,
         0.2601204 , -0.05053168],
       [-0.38715612, -0.18796848, -0.39001609, ...,  0.05300313,
        -0.43507903, -0.04266892],
       [-0.41253905, -0.38603944, -0.49209991, ..., -0.09086834,
        -0.51874055, -0.04582863],
       ...,
       [-1.26999434, -1.70718122, -1.47178029, ...,  1.63154298,
         0.43713452,  0.06107443],
       [-1.41198235, -1.97467349, -1.64126142, ...,  2.25371741,
         1.26774928,  0.16929352],
       [-1.5216273 , -2.24981147, -1.79695419, ..., -0.73707852,
        -2.27655592,  0.17552334]])